 ## Problem description

PetFinder.my uses a basic Cuteness Meter to rank pet photos. It analyzes picture composition and other factors compared to the performance of thousands of pet profiles. 

While this basic tool is helpful, it's still in an experimental stage and the algorithm could be improved. The participants needs to build an AI model using provided data to help make the tool better.  

**Task** 

The task is to predict engagement with a pet's profile( **Pawpularity** ) based on the photograph for that profile. 

**Data** 

The dataset for this competition comprises both images and tabular data(hand-labelled metadata for each photo). 

The train set contains 9912 pet photos 

The test set contains 8 pet photos
> NOTE: The actual test data comprises about **6800** pet photos similar to the training set photos. 


####  **Previous Notebooks**: 
1. [*Understanding the problem & EDA*](https://www.kaggle.com/vivmankar/understanding-the-problem-eda) 
2. [*ML RandomForestRegressor*](https://www.kaggle.com/vivmankar/ml-randomforestregressor)
3. [*CNN_Regressor_using_Transfer_Learning_+_tf.data*](https://www.kaggle.com/vivmankar/cnn-regressor-using-transfer-learning-tf-data)

## Overview of the Notebook

In this notebook we will discuss the Transfer Learning + Multi-input custom model approch to the problem 

#### Data preprocessing

>   1. Analyze the datset 
>   2. Create the dataset with two inputs and one output (tf.data.dataset ) 
>   2. Batching ( To speedup the treaning ) 
>   3. Configure the dataset for performance ( To speedup the treaning )
 
#### Model Building 

>   1. Load the base model ( DenseNet ) 
>   2. Develope image and tabular data models
>   2. Develope a final custom model class  
>   3. Update last layer activation to ReLU(max_value = 100 ) // this helps improving performence 
>   4. Compile model and add callbacks ( Save-Checkpoint, Early Stopping ) 
>   5. Train Model ( MAE on validation split : 13.5925  ) 


## Set up

In [ ]:
import numpy as np
import pandas as pd 

import matplotlib.pyplot as plt 
import seaborn as sns 

import os 
import cv2
import random
 
import tensorflow as tf
from sklearn.model_selection import train_test_split 

import warnings 
warnings.filterwarnings("ignore")

In [ ]:
print("TensorFlow version: {}".format(tf.__version__))
print("Eager execution: {}".format(tf.executing_eagerly()))

In [ ]:
sns.set(style= 'darkgrid', 
       color_codes=True,
       font = 'Arial',
       font_scale= 1.5,
       rc={'figure.figsize':(12,8)})

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>div.output_scroll { height: 70em; }</style>"))

## Data 

In [ ]:
data_dir = "../input/petfinder-pawpularity-score/train/"
test_dir = "../input/petfinder-pawpularity-score/test/"

data = pd.read_csv('../input/petfinder-pawpularity-score/train.csv')
test = pd.read_csv('../input/petfinder-pawpularity-score/test.csv')
ss = pd.read_csv('../input/petfinder-pawpularity-score/sample_submission.csv')

In [ ]:
print(data.shape)
print(test.shape)
print(ss.shape)

In [ ]:
data.head()

### Analyze the data

In [ ]:
_, axs = plt.subplots( 2, 2, figsize=(15, 12))

axs = axs.flatten()
col = data.columns.tolist() 

for a, ax in zip(data.sample(4).iterrows(), axs):
    img = cv2.imread(data_dir + f'{a[1][0]}.jpg')
    img = cv2.resize(img, (600, 600))
    other_info = [col[i] for i in range(13) if a[1][i] == 1 ]
    ax.grid(False)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.imshow(img)
    ax.set_title(f'Id: {a[0]}, Pawpularity : {a[1][13]}, ' + ", ".join(other_info), fontsize= 12, fontweight='bold' )
    
plt.show()

In [ ]:
sns.distplot(data["Pawpularity"])
plt.title("Distribution of Pawpularity")

### Data preprocessing

In [ ]:
train,val  = train_test_split( data, test_size=0.2)  

In [ ]:
train.shape

In [ ]:
val.shape

In [ ]:
train.columns

In [ ]:
filenames = tf.constant(train.Id.map(lambda x : data_dir + f'{x}.jpg' ).tolist())
featurs = tf.constant(train[['Subject Focus', 'Eyes', 'Face', 'Near', 'Action', 'Accessory','Group', 'Collage', 'Human', 'Occlusion', 'Info', 'Blur']])
labels = tf.constant( train.Pawpularity.tolist())

dataset = tf.data.Dataset.from_tensor_slices(( {"input_1": filenames, "input_2": featurs }, labels))

In [ ]:

val_filenames = tf.constant(val.Id.map(lambda x : data_dir + f'{x}.jpg' ).tolist())
val_featurs = tf.constant(val[['Subject Focus', 'Eyes', 'Face', 'Near', 'Action', 'Accessory','Group', 'Collage', 'Human', 'Occlusion', 'Info', 'Blur']])
val_labels = tf.constant( val.Pawpularity.tolist() )


val_dataset = tf.data.Dataset.from_tensor_slices(({"input_1": val_filenames, "input_2": val_featurs }, val_labels))

In [ ]:
list(dataset.as_numpy_iterator())[:5]

In [ ]:
### Hyperparams 

BATCH_SIZE = 64
IMG_SIZE = ( 224 ,  224) 

### File names to images

In [ ]:
def _parse_function( inputs,  output):

    filename = inputs["input_1"]

    image_string = tf.io.read_file(filename)
    image_decoded = tf.image.decode_jpeg(image_string)
    image_resized = tf.image.resize(image_decoded, IMG_SIZE)

    inputs["input_1"] = image_resized

    return  inputs, output

In [ ]:
dataset = dataset.map(_parse_function)
val_dataset = val_dataset.map(_parse_function)

In [ ]:
# Print one key val pair 
def print_pair( input, output):

    image = input["input_1"]
    feature = input["input_2"]

    print(feature.numpy())
    print(feature.numpy().shape)

    plt.figure()
    plt.imshow((image.numpy()).astype(np.uint8))
    plt.title( output.numpy())
    plt.axis('off')
    plt.show()
    print("\n\n\n")


for input, output  in dataset.take(2):
    print_pair(input, output)


### Perfomence Optimization 

In [ ]:
dataset = dataset.batch(BATCH_SIZE) 
val_dataset = val_dataset.batch(BATCH_SIZE) 

 
* **Dataset.cache** keeps the images in memory after they're loaded off disk during the first epoch. This will ensure the dataset does not become a bottleneck while training your model. If your dataset is too large to fit into memory, you can also use this method to create a performant on-disk cache.
* **Dataset.prefetch** overlaps data preprocessing and model execution while training.
 

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE
dataset = dataset.cache().shuffle(500).prefetch(buffer_size=AUTOTUNE)
val_dataset = val_dataset.cache().prefetch(buffer_size=AUTOTUNE) ## We dont need to shuffel the validation data 

## Model

### Download base model 

We will use DenseNet121 as a base model, other opctions for pretrained models can be found [here](https://keras.io/api/applications/)

In [ ]:
base_image_model = tf.keras.applications.DenseNet121( 
                                               include_top=False,
                                               weights='imagenet'
                                               )

In [ ]:
base_image_model.trainable = False

In [ ]:
class ProcessImageBlock(tf.keras.Model):

    def __init__(self):

        super(ProcessImageBlock, self).__init__()

        self.input_l = tf.keras.layers.InputLayer( input_shape = IMG_SIZE + (3,)  ) 
        self.base_model = base_image_model
        self.preprocess_input = tf.keras.applications.densenet.preprocess_input 
        
        self.data_augmentation = tf.keras.Sequential([
                                tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal'),
                                tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
                                ])
        self.rescale = tf.keras.layers.experimental.preprocessing.Rescaling(1./127.5, offset= -1)
        self.gap = tf.keras.layers.GlobalAveragePooling2D() ##  ( batch_size , 2048 )

        self.activation = tf.keras.layers.ReLU()
        self.dense = tf.keras.layers.Dense(512, activation= self.activation )
        self.final = tf.keras.layers.Dense(64, activation= self.activation )

        
    def call(self, input_tensor):

        x = self.input_l(input_tensor)
        x = self.data_augmentation(x)
        x = self.preprocess_input(x)
        x = self.base_model(x, training=False)
        x = self.gap(x)

        x = self.dense(x)
        x = self.final(x)
 
        return  x

class ProcessTabBlock(tf.keras.Model):

    def __init__(self):

        super(ProcessTabBlock, self).__init__()

        self.input_l = tf.keras.layers.InputLayer( input_shape = (12,)  ) 
        self.layer_1 = tf.keras.layers.Dense(32, activation='relu')
        self.layer_2 = tf.keras.layers.Dense(64, activation='relu')
        
    def call(self, input_tensor ):
        
        x = self.input_l(input_tensor)
        x = self.layer_1(x)
        x = self.layer_2(x)

        return x


In [ ]:
class MyCustomModel(tf.keras.Model):

    def __init__(self):

        super(MyCustomModel, self).__init__()

        self.process_image_data = ProcessImageBlock()
        self.process_tabular_data = ProcessTabBlock()

        self.activation_1 = tf.keras.layers.LeakyReLU( alpha=0.3)
        self.activation_2 = tf.keras.layers.ReLU()
        self.activation_final = tf.keras.layers.ReLU(max_value = 100 )
        self.dropout = tf.keras.layers.Dropout(0.2) 

        self.dense_1 =   tf.keras.layers.Dense(64,activation= self.activation_1  )
        self.dense_2 =   tf.keras.layers.Dense(8,activation=  self.activation_2  )
        self.final =   tf.keras.layers.Dense(1, activation=  self.activation_final )
    
    def call(self, inputs ): 

        image = inputs["input_1"]
        feature = inputs["input_2"]

        x1 = self.process_image_data(image)
        x2 = self.process_tabular_data(feature)

        x = tf.keras.layers.concatenate([x1, x2])## ( batch_size, 128 )

        x = self.dense_1(x)
        x = self.dropout(x)
        x = self.dense_2(x)

        x = self.final(x)
   
        return  x

### Compile and Train

In [ ]:
def create_model():
    
    model = MyCustomModel()
    
    model.compile(
        optimizer='adam', 
        loss="mse", # Mean squared error 
        metrics=["mae"] # Mean Absolute Error
      )
    
    return model 

In [ ]:
model = create_model()

In [ ]:
epochs = 20

checkpoint_path = "cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)


# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

es_callback = tf.keras.callbacks.EarlyStopping(
                                monitor='val_mae',
                                patience=3,
                                verbose=1,
                                restore_best_weights=True)

history = model.fit(
                    dataset,
                    validation_data = val_dataset, 
                    epochs=epochs,
                    callbacks = [cp_callback , es_callback ] ,
                    )

### Load the saved model 

In [ ]:
saved_checkpoint_path = "cp.ckpt"

In [ ]:
# Create a basic model instance
new_model = create_model()

# Loads the weights
new_model.load_weights(saved_checkpoint_path)

## Predict on test data

In [ ]:
def _parse_function_test( inputs):

    filename = inputs["input_1"]

    image_string = tf.io.read_file(filename)
    image_decoded = tf.image.decode_jpeg(image_string)
    image_resized = tf.image.resize(image_decoded, IMG_SIZE)

    inputs["input_1"] = image_resized

    return  inputs

In [ ]:

test_filenames = tf.constant(test.Id.map(lambda x : test_dir + f'{x}.jpg' ).tolist())
test_featurs = tf.constant(test[['Subject Focus', 'Eyes', 'Face', 'Near', 'Action', 'Accessory','Group', 'Collage', 'Human', 'Occlusion', 'Info', 'Blur']])
 
test_dataset = tf.data.Dataset.from_tensor_slices(({"input_1": test_filenames, "input_2": test_featurs }))

In [ ]:
test_dataset = test_dataset.map(_parse_function_test)

In [ ]:
test_dataset = test_dataset.batch(len(test))

In [ ]:
predictions = new_model.predict(test_dataset)

In [ ]:
ss.head()

In [ ]:
submission = pd.DataFrame()
submission["Id"] = test["Id"]
submission["Pawpularity"]= predictions

In [ ]:
submission.head()

In [ ]:
ss.columns.equals(submission.columns)

In [ ]:
submission.to_csv('submission.csv', index=False)

## Conclusion:

> #### In the [*last notebook*](https://www.kaggle.com/vivmankar/cnn-regressor-using-transfer-learning-tf-data) we saw the approach that uses only image data, the test_mae was 15.1742, 
> #### while in this approach we have achieved the test_mae of **13.5925** on the same test split, which is an improvement.